# Simple Impementation of E(n) Equivariant Graph Neural Networks

Original paper https://arxiv.org/pdf/2102.09844.pdf by Victor Garcia Satorras, Emiel Hoogeboom, Max Welling

In [3]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# Load QM9 Dataset

In [2]:
!git clone https://github.com/senya-ashukha/simple-equivariant-gnn.git
%cd simple-equivariant-gnn

Cloning into 'simple-equivariant-gnn'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 84 (delta 35), reused 31 (delta 5), pack-reused 0
Receiving objects: 100% (84/84), 175.01 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/Users/ars/Downloads/simple-equivariant-gnn-main/simple-equivariant-gnn


In [4]:
# QM9 is a dataset for Molecular Property Predictions http://quantum-machine.org/datasets/
# We will predict Highest occupied molecular orbital energy 
# https://en.wikipedia.org/wiki/HOMO_and_LUMO
# We use data loaders from the official repo

from qm9.data_utils import get_data, BatchGraph
train_loader, val_loader, test_loader, charge_scale = get_data(num_workers=1)

/Users/allen/Desktop/CS260FinalProject/simple-equivariant-gnn/qm9/data/prepare/process.py:126: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if line[0] is '#':
/Users/allen/Desktop/CS260FinalProject/simple-equivariant-gnn/qm9/data/prepare/process.py:128: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if line_counter is 0:
/Users/allen/Desktop/CS260FinalProject/simple-equivariant-gnn/qm9/data/prepare/process.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif line_counter is 1:
/Users/allen/Desktop/CS260FinalProject/simple-equivariant-gnn/qm9/data/prepare/process.py:143: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(split) is 4:


dict_keys([0, 1, 6, 7, 8, 9])
dict_keys([0, 1, 6, 7, 8, 9])
dict_keys([0, 1, 6, 7, 8, 9])


In [9]:
next(iter(train_loader)).keys()

dict_keys(['num_atoms', 'charges', 'positions', 'index', 'A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv', 'omega1', 'zpve_thermo', 'U0_thermo', 'U_thermo', 'H_thermo', 'G_thermo', 'Cv_thermo', 'one_hot', 'atom_mask', 'edge_mask'])

# Graph Representation

In [3]:
batch = BatchGraph(iter(train_loader).next(), False, charge_scale)
batch

In the batch: num_graphs 96 num_nodes 1759
> .h 		 a tensor of nodes representations 		shape 1759 x 15
> .x 		 a tensor of nodes positions  			shape 1759 x 3
> .edges 	 a tensor of edges, a fully connected graph 	shape 31288 x 2
> .batch  	 a tensor of graph_ids for each node 		tensor([ 0,  0,  0,  ..., 95, 95, 95])

# Define Equivariant Graph Convs  & GNN

In [13]:
def index_sum(agg_size, source, idx, cuda):
    """
        source is N x hid_dim [float]
        idx    is N           [int]
        
        Sums the rows source[.] with the same idx[.];
    """
    print(agg_size, source, idx)
    tmp = torch.zeros((agg_size, source.shape[1]))
    tmp = tmp.cuda() if cuda else tmp
    res = torch.index_add(tmp, 0, idx, source)
    print(res)
    return res

    # (self, dim, index, source)
    # Tensor.index_add_(dim, index, source, *, alpha=1)
    # self[index[i], :, :] += 1 * src[i, :, :]  # if dim == 0
    # essentially aggregating along idx

In [14]:
class ConvEGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        self.cuda = cuda
        
        # computes messages based on hidden representations -> [0, 1]
        self.f_e = nn.Sequential(
            nn.Linear(in_dim*2+1, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim), nn.SiLU())
        
        # predicts "soft" edges based on messages 
        self.f_inf = nn.Sequential( 
            nn.Linear(hid_dim, 1),
            nn.Sigmoid()) 
        
        # updates hidden representations -> [0, 1]
        self.f_h = nn.Sequential(
            nn.Linear(in_dim+hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim))
    
    def forward(self, b):
        e_st, e_end = b.edges[:,0], b.edges[:,1]
        dists = torch.norm(b.x[e_st] - b.x[e_end], dim=1).reshape(-1, 1) # Euclidean distance between the start and end edge
        
        # compute messages
        tmp = torch.hstack([b.h[e_st], b.h[e_end], dists])
        m_ij = self.f_e(tmp) # message at position ij
        
        # predict edges
        e_ij = self.f_inf(m_ij) # edge prediction at position ij
        
        # average e_ij-weighted messages  
        # m_i is num_nodes x hid_dim
        m_i = index_sum(b.h.shape[0], e_ij*m_ij, b.edges[:,0], self.cuda) # average message at position ij
        
        # update hidden representations
        b.h += self.f_h(torch.hstack([b.h, m_i])) # backprop

        return b

In [15]:
class NetEGNN(nn.Module):
    def __init__(self, in_dim=15, hid_dim=128, out_dim=1, n_layers=7, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        
        self.emb = nn.Linear(in_dim, hid_dim) 

        self.gnn = [ConvEGNN(hid_dim, hid_dim, cuda=cuda) for _ in range(n_layers)]
        self.gnn = nn.Sequential(*self.gnn)
        
        # encoder
        self.pre_mlp = nn.Sequential(
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim))
        
        # decoder
        self.post_mlp = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, out_dim))

        if cuda: self.cuda()
        self.cuda = cuda
    
    def forward(self, b):
        b.h = self.emb(b.h)
        
        b = self.gnn(b)
        h_nodes = self.pre_mlp(b.h)
        
        # h_graph is num_graphs x hid_dim
        # h_nodes provides an embedding of all graphs 
        # index_sum provides an aggregate of all graphs 
        h_graph = index_sum(b.nG, h_nodes, b.batch, self.cuda) 
        
        out = self.post_mlp(h_graph)
        return out

In [16]:
epochs = 1000
cuda = True

model = NetEGNN(n_layers=7, cuda=cuda)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, verbose=False)

AssertionError: Torch not compiled with CUDA enabled

# Training

In [ ]:
print('> start training')

tr_ys = train_loader.dataset.data['homo'] 
me, mad = torch.mean(tr_ys), torch.mean(torch.abs(tr_ys - torch.mean(tr_ys)))

if cuda:
    me = me.cuda()
    mad = mad.cuda()

train_loss = []
val_loss = []
test_loss = []

for epoch in range(epochs):
    print('> epoch %s:' % str(epoch).zfill(3), end=' ', flush=True) 
    start = time.time()

    batch_train_loss = []
    batch_val_loss = []
    batch_test_loss = []

    model.train()
    for batch in train_loader:
        batch = BatchGraph(batch, cuda, charge_scale)
        
        out = model(batch).reshape(-1)
        loss =  F.l1_loss(out,  (batch.y-me)/mad)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            loss =  F.l1_loss(out*mad+me, batch.y)

        batch_train_loss += [float(loss.data.cpu().numpy())]  
        
    train_loss += [np.mean(batch_train_loss)/0.001]
    
    print('train %.3f' % train_loss[-1], end=' ', flush=True)
    
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_val_loss += [np.mean(loss)]
            
        val_loss += [np.mean(batch_val_loss)/0.001]
        
        print('val %.3f' % val_loss[-1], end=' ', flush=True)
        
        for batch in test_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_test_loss += [np.mean(loss)]

        test_loss += [np.mean(batch_test_loss)/0.001]
        
    end = time.time()

    print('test %.3f (%.1f sec)' % (test_loss[-1], end-start), flush=True)
    lr_scheduler.step()

> start training
> epoch 000: 

In [ ]:
# > start training 
# > epoch 000: train 264.008 val 243.351 test 242.635 (94.3 sec)
# > epoch 001: train 211.893 val 211.575 test 210.144 (92.8 sec)
# > epoch 002: train 185.362 val 164.960 test 165.087 (93.9 sec)
# > epoch 003: train 163.121 val 150.953 test 150.533 (93.2 sec)
# ...
# > epoch 998: train 0.032 val 30.157 test 30.886 (93.4 sec)
# > epoch 999: train 0.032 val 30.157 test 30.886 (93.4 sec)